In [3]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer='dm2_dataset_2425_imdb/preprocessed_full.csv')
df

,originalTitle,rating,startYear,endYear,runtimeMinutes,awardWins,numVotes,worstRating,bestRating,totalImages,...,genres,castNumber,companiesNumber,averageRating,regions,externalLinks,writerCredits,directorsCredits,soundMixes,quotesTotal
0,Carmencita,"(5, 6]",1894,1894.0,1.0,0,2089,1,10,2,...,"['Documentary', 'Short']",1,3,5.7,"['DE', 'US', 'HU', 'GR', 'RU', 'UA', 'JP']",9,0,1,['silent'],0
1,Un bon bock,"(5, 6]",1892,1892.0,12.0,0,183,1,10,2,...,"['Animation', 'Short']",0,0,5.4,"['HU', 'DE', 'FR', 'RO', 'RU', 'JP']",3,0,1,['silent'],0
2,Chinese Opium Den,"(4, 5]",1894,1894.0,1.0,0,195,1,10,1,...,['Short'],0,1,5.0,"['US', 'DE', 'PT', 'HU', 'RU']",4,0,1,['silent'],0
3,Edison Kinetoscopic Record of a Sneeze,"(5, 6]",1894,1894.0,1.0,1,2237,1,10,3,...,"['Documentary', 'Short']",1,6,5.4,"['UA', 'JP', 'HU', 'US', 'DE', 'RU']",6,0,1,['silent'],0
4,L'arrivée d'un train à La Ciotat,"(7, 8]",1896,1896.0,1.0,0,13115,1,10,12,...,"['Documentary', 'Short']",6,5,7.4,"['HU', 'DE', 'TR', 'FI', 'FR', 'IT', 'ES', 'SK...",21,0,2,['silent'],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,Nuestra película,"(6, 7]",1993,1993.0,96.0,0,11,1,10,0,...,['Documentary'],0,2,7.0,['CO'],1,1,1,[],0
149527,Eco,"(6, 7]",2019,2019.0,14.0,0,15,1,10,4,...,"['Drama', 'Short']",6,3,6.4,['ES'],5,1,1,[],0
149528,Women Take Center Stage,"(5, 6]",2019,2019.0,NaN,0,12,1,10,2,...,['Reality-TV'],1,0,5.6,[],0,0,0,[],0
149529,Horrid Henry and the Christening Crisis,"(7, 8]",2011,2011.0,10.0,0,9,1,10,1,...,"['Adventure', 'Animation', 'Comedy']",7,0,7.1,[],0,3,1,[],0


In [5]:
df['genres'].unique()

array(["['Documentary', 'Short']", "['Animation', 'Short']", "['Short']",
       ..., "['Drama', 'Mystery', 'Reality-TV']",
       "['Adventure', 'Animation', 'Romance']",
       "['Adult', 'Reality-TV', 'Romance']"], dtype=object)

In [5]:
from support.conversion import convert_string_list, feats_to_list

for feat in feats_to_list:
    df[feat] = df[feat].apply(convert_string_list)
    
df

,originalTitle,rating,startYear,endYear,runtimeMinutes,awardWins,numVotes,worstRating,bestRating,totalImages,...,genres,castNumber,companiesNumber,averageRating,regions,externalLinks,writerCredits,directorsCredits,soundMixes,quotesTotal
0,Carmencita,"(5, 6]",1894,1894.0,1.0,0,2089,1,10,2,...,"[Documentary, Short]",1,3,5.7,"[DE, US, HU, GR, RU, UA, JP]",9,0,1,[silent],0
1,Un bon bock,"(5, 6]",1892,1892.0,12.0,0,183,1,10,2,...,"[Animation, Short]",0,0,5.4,"[HU, DE, FR, RO, RU, JP]",3,0,1,[silent],0
2,Chinese Opium Den,"(4, 5]",1894,1894.0,1.0,0,195,1,10,1,...,[Short],0,1,5.0,"[US, DE, PT, HU, RU]",4,0,1,[silent],0
3,Edison Kinetoscopic Record of a Sneeze,"(5, 6]",1894,1894.0,1.0,1,2237,1,10,3,...,"[Documentary, Short]",1,6,5.4,"[UA, JP, HU, US, DE, RU]",6,0,1,[silent],0
4,L'arrivée d'un train à La Ciotat,"(7, 8]",1896,1896.0,1.0,0,13115,1,10,12,...,"[Documentary, Short]",6,5,7.4,"[HU, DE, TR, FI, FR, IT, ES, SK, PL, GB, XWW, ...",21,0,2,[silent],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,Nuestra película,"(6, 7]",1993,1993.0,96.0,0,11,1,10,0,...,[Documentary],0,2,7.0,[CO],1,1,1,[],0
149527,Eco,"(6, 7]",2019,2019.0,14.0,0,15,1,10,4,...,"[Drama, Short]",6,3,6.4,[ES],5,1,1,[],0
149528,Women Take Center Stage,"(5, 6]",2019,2019.0,NaN,0,12,1,10,2,...,[Reality-TV],1,0,5.6,[],0,0,0,[],0
149529,Horrid Henry and the Christening Crisis,"(7, 8]",2011,2011.0,10.0,0,9,1,10,1,...,"[Adventure, Animation, Comedy]",7,0,7.1,[],0,3,1,[],0


In [6]:
df['runtimeMinutes']

0          1.0
1         12.0
2          1.0
3          1.0
4          1.0
          ... 
149526    96.0
149527    14.0
149528     NaN
149529    10.0
149530     NaN
Name: runtimeMinutes, Length: 149531, dtype: float64

In [7]:
# impute missing runtimeMinutes
from support.imputation import impute_runtime_minutes

# Assuming `impute_runtime_minutes(df)` returns a Series with the same second-level index
df['runtimeMinutes'] = impute_runtime_minutes(df)
df['runtimeMinutes']

0          1.0
1         12.0
2          1.0
3          1.0
4          1.0
          ... 
149526    96.0
149527    14.0
149528    30.0
149529    10.0
149530    45.0
Name: runtimeMinutes, Length: 149531, dtype: float64

In [8]:
from sklearn.ensemble import IsolationForest

# Select numerical columns for outlier detection
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Initialize Isolation Forest
iso_forest = IsolationForest(random_state=42)

# Fit the model and predict outliers
df['outlier'] = iso_forest.fit_predict(df[numerical_columns])

# Mark outliers (-1) and inliers (1)
df['outlier'] = df['outlier'].map({1: 'inlier', -1: 'outlier'})

df[['outlier']].value_counts()

ValueError: Input X contains NaN.
IsolationForest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values